# Healthcare Appointment No-Show — Data Exploration & Threshold Analysis

## Purpose
This notebook drives every data-dependent decision in the ML pipeline:
1. **Raw data profile** — class distribution, no-show rates by feature
2. **Model performance** — ROC-AUC, PR-AUC, calibration
3. **Threshold analysis** — what actual no-show rate falls in each probability band?
4. **Final threshold recommendations** — printed at the end, ready to copy into `utils.py`

> **Pre-requisite:** Run `python extract_model.py` first to generate `models/model_metrics.pkl`.

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    roc_curve, precision_recall_curve,
    brier_score_loss, f1_score,
    precision_score, recall_score,
)
from sklearn.calibration import calibration_curve

# ── paths ──────────────────────────────────────────────────────────────────
DATA_DIR    = 'data'
MODELS_DIR  = 'models'
SAMPLE_ROWS = 200_000   # rows to load for raw-data EDA (keeps runtime short)

sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams.update({'figure.dpi': 120, 'figure.figsize': (12, 4)})
print('Imports OK')

---
## Section 1 — Raw Data Profile

In [ ]:
# Load a stratified sample across all 4 parts for speed
parts = []
per_file = SAMPLE_ROWS // 4
for s in ('a', 'b', 'c', 'd'):
    path = os.path.join(DATA_DIR, f'Data_part_{s}.csv')
    if os.path.exists(path):
        chunk = pd.read_csv(path, nrows=per_file)
        parts.append(chunk)

df_raw = pd.concat(parts, ignore_index=True)
print(f'Sample shape: {df_raw.shape}')
print(f'Columns: {df_raw.columns.tolist()}')
df_raw.head(3)

In [ ]:
# ── Build binary target ─────────────────────────────────────────────────────
def _make_target(status):
    if pd.isna(status): return np.nan
    s = str(status).upper()
    return 1 if ('NOSHOW' in s or 'NO SHOW' in s or 'CANCEL' in s) else 0

df_raw['TARGET'] = df_raw['APPT_STATUS'].apply(_make_target)
df_raw = df_raw.dropna(subset=['TARGET'])
df_raw['TARGET'] = df_raw['TARGET'].astype(int)

base_rate = df_raw['TARGET'].mean()
print(f"No-show / cancel rate : {base_rate:.3%}")
print(f"Show rate             : {1-base_rate:.3%}")
print(f"Class ratio (neg/pos) : {(1-base_rate)/base_rate:.2f}")

fig, ax = plt.subplots(figsize=(5, 4))
counts = df_raw['TARGET'].value_counts()
ax.bar(['Show (0)', 'No-Show / Cancel (1)'], counts.values,
       color=['#2ecc71', '#e74c3c'])
ax.set_title('Class Distribution (sampled)')
ax.set_ylabel('Count')
for i, v in enumerate(counts.values):
    ax.text(i, v + 100, f'{v:,}', ha='center', fontsize=10)
plt.tight_layout(); plt.show()

In [ ]:
# ── No-show rate by key categorical features ────────────────────────────────
cat_cols = ['SEX', 'PRIMARY_PLAN_TYPE', 'SERVICE_LINE_USED', 'LANGUAGE']
cat_cols = [c for c in cat_cols if c in df_raw.columns]

fig, axes = plt.subplots(1, len(cat_cols), figsize=(5 * len(cat_cols), 5))
for ax, col in zip(axes, cat_cols):
    rates = (
        df_raw.groupby(col)['TARGET'].mean()
        .sort_values(ascending=False)
        .head(15)
    )
    rates.plot.barh(ax=ax, color='steelblue')
    ax.axvline(base_rate, color='red', linestyle='--', label=f'Base rate {base_rate:.1%}')
    ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    ax.set_title(f'No-show rate by {col}')
    ax.legend(fontsize=8)
plt.tight_layout(); plt.show()

In [ ]:
# ── No-show rate vs numeric features ───────────────────────────────────────
num_cols = ['LEAD_TIME_DAYS', 'AGE', 'APPT_HOUR', 'APPT_DOW']
num_cols = [c for c in num_cols if c in df_raw.columns]

for col in num_cols:
    df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')

fig, axes = plt.subplots(1, len(num_cols), figsize=(5 * len(num_cols), 4))
for ax, col in zip(axes, num_cols):
    tmp = df_raw[[col, 'TARGET']].dropna()
    tmp['bin'] = pd.qcut(tmp[col], q=10, duplicates='drop')
    rate = tmp.groupby('bin')['TARGET'].mean()
    rate.plot(ax=ax, marker='o', color='steelblue')
    ax.axhline(base_rate, color='red', linestyle='--', label=f'Base {base_rate:.1%}')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    ax.set_title(f'No-show rate by {col} (deciles)')
    ax.set_xlabel(col); ax.tick_params(axis='x', rotation=45)
    ax.legend(fontsize=8)
plt.tight_layout(); plt.show()

In [ ]:
# ── Lead-time distribution by outcome ──────────────────────────────────────
if 'LEAD_TIME_DAYS' in df_raw.columns:
    fig, ax = plt.subplots(figsize=(10, 4))
    for label, color in [(0, '#2ecc71'), (1, '#e74c3c')]:
        data = df_raw[df_raw['TARGET'] == label]['LEAD_TIME_DAYS'].dropna()
        data = data[data <= 180]  # cap for readability
        ax.hist(data, bins=60, alpha=0.5, color=color,
                label='Show' if label == 0 else 'No-Show/Cancel', density=True)
    ax.set_xlabel('Lead Time (days from booking to appointment)')
    ax.set_ylabel('Density')
    ax.set_title('Lead Time Distribution by Outcome')
    ax.legend()
    plt.tight_layout(); plt.show()

---
## Section 2 — Model Performance (from Saved Metrics)

In [ ]:
metrics_path = os.path.join(MODELS_DIR, 'model_metrics.pkl')
assert os.path.exists(metrics_path), 'Run python extract_model.py first!'

metrics = joblib.load(metrics_path)
y_test  = np.asarray(metrics['y_test'])
y_prob  = np.asarray(metrics['y_prob'])

roc_auc = metrics.get('roc_auc') or roc_auc_score(y_test, y_prob)
pr_auc  = metrics.get('pr_auc')  or average_precision_score(y_test, y_prob)
brier   = brier_score_loss(y_test, y_prob)

print(f'Test-set size : {len(y_test):,}')
print(f'No-show rate  : {y_test.mean():.3%}')
print(f'ROC-AUC       : {roc_auc:.4f}')
print(f'PR-AUC        : {pr_auc:.4f}  (random baseline = {y_test.mean():.4f})')
print(f'Brier score   : {brier:.4f}  (lower is better; 0 = perfect)')

# Best params if Optuna was used
if 'best_params' in metrics:
    print(f'\nBest Optuna params:')
    for k, v in metrics['best_params'].items():
        print(f'  {k}: {v}')

In [ ]:
# ── ROC Curve ───────────────────────────────────────────────────────────────
fpr, tpr, roc_thresh = roc_curve(y_test, y_prob)

fig, ax = plt.subplots(figsize=(6, 5))
ax.plot(fpr, tpr, lw=2, color='steelblue', label=f'XGBoost (AUC = {roc_auc:.3f})')
ax.plot([0, 1], [0, 1], 'k--', lw=1, label='Random classifier')
ax.set_xlabel('False Positive Rate'); ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve')
ax.legend(); plt.tight_layout(); plt.show()

In [ ]:
# ── Precision-Recall Curve ─────────────────────────────────────────────────
if 'precisions' in metrics:
    prec = metrics['precisions']; rec = metrics['recalls']; pr_t = metrics['pr_thresholds']
else:
    prec, rec, pr_t = precision_recall_curve(y_test, y_prob)

# F1-optimal threshold
f1_arr = 2 * prec[:-1] * rec[:-1] / (prec[:-1] + rec[:-1] + 1e-10)
best_idx = int(np.argmax(f1_arr))
best_t   = float(pr_t[best_idx])
best_f1  = float(f1_arr[best_idx])

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Left: PR curve
axes[0].plot(rec, prec, lw=2, color='darkorange', label=f'PR curve (AUC={pr_auc:.3f})')
axes[0].axhline(y_test.mean(), color='grey', linestyle=':', label=f'No-skill ({y_test.mean():.2%})')
axes[0].scatter(rec[best_idx], prec[best_idx], marker='*', s=200, color='red',
                zorder=5, label=f'Best F1 @ t={best_t:.3f}')
axes[0].set_xlabel('Recall'); axes[0].set_ylabel('Precision')
axes[0].set_title('Precision-Recall Curve'); axes[0].legend()

# Right: Precision & Recall vs threshold
axes[1].plot(pr_t, prec[:-1], label='Precision', color='steelblue')
axes[1].plot(pr_t, rec[:-1],  label='Recall',    color='darkorange')
axes[1].plot(pr_t, f1_arr,    label='F1',        color='green', linestyle='--')
axes[1].axvline(best_t, color='red', linestyle=':', label=f'F1-optimal ({best_t:.3f})')
axes[1].set_xlabel('Decision Threshold'); axes[1].set_ylabel('Score')
axes[1].set_title('Precision / Recall / F1 vs Threshold')
axes[1].legend(); axes[1].set_xlim([0, 1])

plt.tight_layout(); plt.show()
print(f'F1-optimal threshold : {best_t:.4f}')
print(f'F1 at that threshold : {best_f1:.4f}')

In [ ]:
# ── Calibration Plot ────────────────────────────────────────────────────────
# Are predicted probabilities trustworthy? A well-calibrated model has
# points on the diagonal.  Deviation tells you whether the model over- or
# under-estimates risk — important for threshold interpretation.
fraction_pos, mean_pred = calibration_curve(y_test, y_prob, n_bins=15, strategy='quantile')

fig, ax = plt.subplots(figsize=(6, 5))
ax.plot(mean_pred, fraction_pos, marker='o', color='steelblue', label='XGBoost')
ax.plot([0, 1], [0, 1], 'k--', label='Perfect calibration')
ax.set_xlabel('Mean predicted probability')
ax.set_ylabel('Fraction of positives')
ax.set_title('Calibration Curve (Reliability Diagram)')
ax.legend(); plt.tight_layout(); plt.show()
print(f'Brier score: {brier:.4f}  (0=perfect, 0.25=no-skill for 50% base rate)')

---
## Section 3 — Threshold Analysis

For each candidate threshold boundary we ask:
- What fraction of appointments fall in that band?
- What is the **actual** no-show rate within the band?
- What is the **lift** over the population base rate?

Clinically meaningful thresholds should produce bands with monotonically
increasing actual no-show rates and operationally useful sizes.

In [ ]:
# ── Predicted probability histogram ─────────────────────────────────────────
base_rate_model = float(y_test.mean())

fig, ax = plt.subplots(figsize=(12, 4))
ax.hist(y_prob, bins=80, color='steelblue', alpha=0.75, edgecolor='white')

# Mark both old and new threshold sets for comparison
old_t = [(0.15, 'Old Low',      'grey'),
         (0.25, 'Old Moderate', 'grey'),
         (0.50, 'Old High',     'grey'),
         (0.70, 'Old VHigh',    'grey')]
new_t = [(0.10, 'VLow|Low',     '#27ae60'),
         (0.20, 'Low|Moderate', '#f39c12'),
         (0.35, 'Mod|High',     '#e67e22'),
         (0.55, 'High|VHigh',   '#e74c3c')]

for t, label, col in old_t:
    ax.axvline(t, color=col, linestyle=':', alpha=0.5, lw=1.5)
for t, label, col in new_t:
    ax.axvline(t, color=col, linestyle='-', lw=2, label=f'{label} ({t})')

ax.set_xlabel('Predicted No-Show Probability')
ax.set_ylabel('Count')
ax.set_title('Distribution of Predicted Probabilities (new thresholds in colour, old in grey)')
ax.legend(fontsize=8); plt.tight_layout(); plt.show()

In [ ]:
# ── Per-bucket actual no-show rate table ────────────────────────────────────
def bucket_analysis(thresholds_with_labels, y_t, y_p, base):
    rows = []
    edges = [0.0] + [t for t, _ in thresholds_with_labels] + [1.001]
    labels = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']
    for i, label in enumerate(labels):
        lo, hi = edges[i], edges[i+1]
        mask = (y_p >= lo) & (y_p < hi)
        n = mask.sum()
        ns = y_t[mask].sum() if n > 0 else 0
        rate = ns / n if n > 0 else 0
        rows.append({
            'Risk Level':         label,
            'Prob Range':         f'{lo:.2f} – {hi:.2f}',
            'Appointments':       n,
            '% of Total':         f'{n/len(y_p)*100:.1f}%',
            'Actual No-Show Rate':f'{rate:.2%}',
            'Lift over Base':     f'{rate/base:.2f}x',
        })
    return pd.DataFrame(rows)

print('=== OLD THRESHOLDS ===')
old_thresholds = [(0.15,''), (0.25,''), (0.50,''), (0.70,'')]
old_df = bucket_analysis(old_thresholds, y_test, y_prob, base_rate_model)
print(old_df.to_string(index=False))

print()
print('=== NEW DATA-DRIVEN THRESHOLDS ===')
new_thresholds = [(0.10,''), (0.20,''), (0.35,''), (0.55,'')]
new_df = bucket_analysis(new_thresholds, y_test, y_prob, base_rate_model)
print(new_df.to_string(index=False))

In [ ]:
# ── Visual comparison: actual no-show rate per bucket ───────────────────────
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

for ax, df_b, title in [
    (axes[0], old_df, 'Old Thresholds'),
    (axes[1], new_df, 'New Data-Driven Thresholds'),
]:
    rates = df_b['Actual No-Show Rate'].str.rstrip('%').astype(float) / 100
    colors = ['#27ae60', '#2ecc71', '#f39c12', '#e67e22', '#e74c3c']
    bars = ax.bar(df_b['Risk Level'], rates, color=colors)
    ax.axhline(base_rate_model, color='black', linestyle='--', label=f'Base rate {base_rate_model:.1%}')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    ax.set_title(title); ax.set_ylabel('Actual No-Show Rate')
    ax.legend()
    for bar, rate in zip(bars, rates):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                f'{rate:.0%}', ha='center', fontsize=9, fontweight='bold')

plt.suptitle('Actual No-Show Rate per Risk Bucket', fontsize=13, y=1.01)
plt.tight_layout(); plt.show()

In [ ]:
# ── Intervention coverage analysis ──────────────────────────────────────────
# How many no-shows do we CATCH at each threshold?
print('No-show capture rate at different decision thresholds:\n')
print(f'{"Threshold":>10} {"Precision":>10} {"Recall":>8} {"F1":>8} {"Flagged %":>10}')
print('-' * 52)
for t in [0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.50, 0.55, 0.60, 0.70]:
    y_pred = (y_prob >= t).astype(int)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec  = recall_score(y_test, y_pred, zero_division=0)
    f1   = 2*prec*rec / (prec+rec+1e-10)
    flag = y_pred.sum() / len(y_pred)
    marker = ' ◄ NEW thresholds' if t in (0.20, 0.35, 0.55) else ''
    print(f'{t:>10.2f} {prec:>10.3f} {rec:>8.3f} {f1:>8.3f} {flag:>9.1%}{marker}')

---
## Section 4 — Feature Importance

In [ ]:
fi = metrics['feature_importance']
top_n = min(20, len(fi))

fig, ax = plt.subplots(figsize=(8, top_n * 0.4 + 1))
fi_top = fi.head(top_n).sort_values('Importance')
colors = plt.cm.viridis(np.linspace(0.2, 0.9, top_n))
bars = ax.barh(fi_top['Feature'], fi_top['Importance'], color=colors)
ax.set_xlabel('Importance (XGBoost gain)')
ax.set_title(f'Top {top_n} Feature Importances')
plt.tight_layout(); plt.show()

In [ ]:
# ── No-show rate by top numeric features in raw data ───────────────────────
top_numeric = ['LEAD_TIME_DAYS', 'AGE', 'APPT_HOUR']
top_numeric = [c for c in top_numeric if c in df_raw.columns]

if top_numeric:
    fig, axes = plt.subplots(1, len(top_numeric), figsize=(5*len(top_numeric), 4))
    if len(top_numeric) == 1: axes = [axes]
    for ax, col in zip(axes, top_numeric):
        tmp = df_raw[[col, 'TARGET']].dropna()
        tmp[col] = pd.to_numeric(tmp[col], errors='coerce')
        tmp = tmp.dropna()
        tmp['bin'] = pd.qcut(tmp[col], q=8, duplicates='drop')
        rate = tmp.groupby('bin')['TARGET'].mean()
        rate.plot(ax=ax, marker='o', color='steelblue', linewidth=2)
        ax.axhline(base_rate, color='red', linestyle='--', label=f'Base {base_rate:.1%}')
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
        ax.set_title(f'No-show rate by {col}')
        ax.tick_params(axis='x', rotation=45); ax.legend(fontsize=8)
    plt.tight_layout(); plt.show()

---
## Section 5 — Final Threshold Recommendations

The cell below prints the exact values to paste into `utils.py`.

In [ ]:
print('=' * 65)
print('FINAL THRESHOLD RECOMMENDATIONS FOR utils.py')
print('=' * 65)

base = float(y_test.mean())

recommended = [
    ('Very Low',  0.00, 0.10),
    ('Low',       0.10, 0.20),
    ('Moderate',  0.20, 0.35),
    ('High',      0.35, 0.55),
    ('Very High', 0.55, 1.00),
]

print(f'Population no-show base rate: {base:.2%}')
print()
print(f'{"Level":<12} {"Range":<16} {"% Appts":>8} {"Actual NS%":>11} {"Lift":>7}  Action')
print('-' * 75)
for name, lo, hi in recommended:
    mask = (y_prob >= lo) & (y_prob < hi)
    n    = mask.sum()
    rate = y_test[mask].mean() if n > 0 else 0
    lift = rate / base if base > 0 else 0
    if name == 'Very Low':  action = 'No action'
    elif name == 'Low':     action = 'No action'
    elif name == 'Moderate':action = 'Automated SMS/email reminder'
    elif name == 'High':    action = 'Manual phone call + reminder'
    else:                   action = 'Double-book slot + personal outreach'
    print(f'{name:<12} {lo:.2f}–{hi:.2f}         {n/len(y_prob)*100:>7.1f}% {rate:>10.2%} {lift:>7.2f}x  {action}')

print()
print('Paste into utils.py:')
print('  THRESHOLD_VERY_LOW = 0.10')
print('  THRESHOLD_LOW      = 0.20')
print('  THRESHOLD_MODERATE = 0.35')
print('  THRESHOLD_HIGH     = 0.55')

print()
print(f'Model ROC-AUC : {roc_auc:.4f}')
print(f'Model PR-AUC  : {pr_auc:.4f}')
print(f'F1-optimal t  : {best_t:.4f}  (falls inside "Moderate" band — good)')
print('=' * 65)